In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv('train.csv')


In [3]:
def add_column(I):
    list_column = I.columns
    ac = 0
    for column in list_column:
        ac = ac + I[column] 
    ac = pd.DataFrame(ac)
    return ac


In [36]:
def mult_column(I):
    I = pd.DataFrame(I)
    list_column = I.columns
    mc = 1
    for column in list_column:
        mc = mc * I[column] 
    mc = pd.DataFrame(mc)
    return mc

In [37]:
def column_cube(I):
    I = pd.DataFrame(I)
    list_column = I.columns
    c3 = 1
    for column in list_column:
        c3 = c3 * I[column]^3 
    c3 = pd.DataFrame(c3)
    return c3

In [39]:
y = df_train['Survived']
X = df_train.loc[:, df_train.columns != 'Survived']


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=45)

In [41]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

In [93]:
age_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='mean')),
    (StandardScaler()),
)
emb_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    (OneHotEncoder(sparse=False, handle_unknown='ignore')),
)
fare_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='mean')),
    (StandardScaler()),
    #(FunctionTransformer(mult_column, validate=False))
)
mult_pipeline = make_pipeline(
    (SimpleImputer(missing_values=np.nan, strategy='mean')),
    (StandardScaler()),
    (FunctionTransformer(mult_column, validate=False)),  
     )

In [94]:
X.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [95]:
transformer = make_column_transformer(
    # (transformer object, column names)
    (age_pipeline, ['Age']),
    (emb_pipeline, ['Embarked']),
    (fare_pipeline, ['Fare']),
    (OneHotEncoder(sparse=False), ['Pclass']),
    (OneHotEncoder(sparse=False), ['Sex']),
    (FunctionTransformer(add_column, validate=False), ['SibSp', 'Parch']),
    #(FunctionTransformer(mult_column, validate=False), ['Pclass', 'Fare']),
    (mult_pipeline, ['Pclass', 'Age']),
    #(mult_pipeline, ['Pclass', 'Sex']),
    #(FunctionTransformer(mult_column, validate=False), ['Pclass', 'Sex']),
    (FunctionTransformer(column_cube, validate=False), ['SibSp']),
    )



In [96]:
transformer.fit(X_train)
X_test_trans = transformer.transform(X_train)
X_test_trans = pd.DataFrame(X_test_trans)
X_test_trans


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-2.725369e-16,0.0,0.0,1.0,-0.475671,0.0,0.0,1.0,0.0,1.0,0.0,-2.290449e-16,3.0
1,-3.761018e-01,0.0,0.0,1.0,-0.491285,0.0,0.0,1.0,0.0,1.0,0.0,-3.160828e-01,3.0
2,-1.373362e+00,1.0,0.0,0.0,-0.413906,0.0,0.0,1.0,0.0,1.0,1.0,-1.154198e+00,2.0
3,2.692391e+00,1.0,0.0,0.0,0.522702,1.0,0.0,0.0,0.0,1.0,1.0,-4.101525e+00,3.0
4,-1.296650e+00,0.0,0.0,1.0,-0.261459,0.0,1.0,0.0,1.0,0.0,1.0,4.427788e-01,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,-7.596634e-01,0.0,0.0,1.0,-0.461518,0.0,0.0,1.0,0.0,1.0,0.0,-6.384349e-01,3.0
619,-2.226772e-01,0.0,0.0,1.0,-0.058400,1.0,0.0,0.0,0.0,1.0,0.0,3.392212e-01,3.0
620,1.541706e+00,1.0,0.0,0.0,1.343166,1.0,0.0,0.0,0.0,1.0,1.0,-2.348599e+00,2.0
621,-2.725369e-16,0.0,0.0,1.0,0.421479,0.0,0.0,1.0,0.0,1.0,0.0,-2.290449e-16,3.0


In [97]:
X_train_trans = transformer.transform(X_train)
X_test_trans = transformer.transform(X_test)

In [98]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_trans, y_train)


LogisticRegression(max_iter=1000)

In [99]:
y_pred = model_lr.predict(X_test_trans)


In [100]:
accuracy_score(y_pred, y_test)

0.835820895522388

In [101]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=1000, max_depth=5, n_jobs=1) # CPUs (-1 = all)

In [102]:
model_rf.fit(X_train_trans, y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=1)

In [103]:
model_rf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(max_depth=5, n_estimators=1000, n_jobs=1)>

In [104]:
y_pred_rf = model_rf.predict(X_test_trans)


In [107]:
accuracy_score(y_pred_rf, y_test)

0.8544776119402985

In [108]:
from sklearn.neighbors import KNeighborsClassifier
model_knn= KNeighborsClassifier(n_neighbors =5)
model_knn.fit(X_train_trans, y_train)
y_pred_knn = model_knn.predict(X_test_trans)
accuracy_score(y_pred_knn, y_test)

0.8059701492537313

In [109]:
from sklearn.svm import SVC, LinearSVC

model_linear_svc = LinearSVC(max_iter=10000)
model_linear_svc.fit(X_train_trans, y_train)
y_pred_svc = model_linear_svc.predict(X_test_trans)
accuracy_score(y_pred_svc, y_test)

0.8432835820895522

In [110]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
Combined_fitting = VotingClassifier(estimators=[
         ('lr', model_lr), ('rf', model_rf), ('knn', model_knn), ('svc', model_linear_svc)], voting='hard')
Combined_fitting = Combined_fitting.fit(X_train_trans, y_train)
y_pred_combined = Combined_fitting.predict(X_test_trans)
accuracy_score(y_pred_combined, y_test)

0.8470149253731343

In [111]:
df_test = pd.read_csv('test.csv')
#X_Kaggle = df_test.drop(['Cabin'], axis=1)
X_Kaggle = df_test


In [112]:
X_Kaggle.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [113]:
X_test_cleaned = transformer.transform(X_Kaggle)
X_test_cleaned=pd.DataFrame(X_test_cleaned)
X_test_cleaned.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
dtype: int64

In [114]:
predictions = model_rf.predict(X_test_cleaned)
predictions

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [116]:
submission = pd.DataFrame({'PassengerId':X_Kaggle['PassengerId'],'Survived':predictions})
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [117]:
filename = 'Titanic_Predictions_Friday1.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: Titanic_Predictions_Friday1.csv
